Notebook to get the extreme predictions' subject name for a given classifier.

In [8]:
import pandas as pd

from contrastive.evaluation.train_multiple_classifiers import compute_auc
from contrastive.utils.models_database import import_bdd

In [9]:
bdd = import_bdd()
bdd

,model_path,auc,auc_std,accuracy,accuracy_std,backbone_name,batch_size,drop_rate,early_stopping_patience,foldlabel,input_size,lr,max_angle,max_epochs,num_outputs,num_representation_features,temperature,temperature_initial,train_loss,val_loss,exclude,0.1_quantile,dataset_name,git_branch
100,/neurospin/dico/agaudin/Runs/04_pointnet/Outpu...,0.752147,0.003479,0.705150,0.009098,convnet,16,0.05,100,True,"(1, 17, 40, 38)",0.0008,6,251,4,4,0.1,0.1,1.240257,1.669160,False,0.078811,cingulate_HCP_half_1,Run_03_aymeric
132,/neurospin/dico/agaudin/Runs/04_pointnet/Outpu...,0.748543,0.003786,0.695589,0.006983,convnet,16,0.05,100,False,"(1, 17, 40, 38)",0.0004,10,251,30,30,0.1,0.1,0.086302,0.153539,False,-0.147380,cingulate_HCP_half_1,Run_03_aymeric
120,/neurospin/dico/agaudin/Runs/04_pointnet/Outpu...,0.743639,0.004698,0.703261,0.006989,convnet,16,0.05,100,True,"(1, 17, 40, 38)",0.0004,6,251,4,4,0.1,0.1,1.251985,1.929024,False,0.594088,cingulate_HCP_half_1,Run_03_aymeric
119,/neurospin/dico/agaudin/Runs/04_pointnet/Outpu...,0.729348,0.002100,0.669314,0.005821,convnet,16,0.05,100,True,"(1, 17, 40, 38)",0.0004,6,251,4,4,0.1,0.1,1.374010,1.463949,False,0.199899,cingulate_HCP_half_1,Run_03_aymeric
129,/neurospin/dico/agaudin/Runs/04_pointnet/Outpu...,0.721823,0.048261,0.631249,0.058619,densenet,16,0.05,100,True,"(1, 17, 40, 38)",0.0004,6,251,30,30,0.1,0.1,0.515141,1.306467,False,0.640115,cingulate_HCP_half_1,Run_03_aymeric
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,/neurospin/dico/agaudin/Runs/03_monkeys/Output...,0.482994,0.034076,0.512551,0.030729,densenet,16,0.15,100,False,"(1, 17, 40, 38)",0.0004,10,250,4,4,0.1,0.1,3.753164,3.651408,False,-0.688383,cingulate_HCP_half_1,Run_03_aymeric
161,/neurospin/dico/agaudin/Runs/04_pointnet/Outpu...,0.478818,0.022703,0.576880,0.003572,pointnet,16,0.05,100,True,"(1, 17, 40, 38)",0.0004,6,250,4,4,0.5,0.5,5.256860,5.389712,False,0.129245,cingulate_HCP_half_1,pointnet
19,/neurospin/dico/agaudin/Runs/03_monkeys/Output...,0.473885,0.042166,0.510393,0.050783,convnet,16,0.15,100,False,"(1, 17, 40, 38)",0.0004,10,250,4,4,0.1,0.1,8.022524,6.433455,False,0.943119,cingulate_HCP_half_1,Run_03_aymeric
38,/neurospin/dico/agaudin/Runs/03_monkeys/Output...,0.466274,0.034258,0.547683,0.020623,densenet,16,0.15,100,False,"(1, 17, 40, 38)",0.0004,10,250,4,4,0.1,0.1,3.534628,3.446562,False,-0.106718,cingulate_HCP_half_1,Run_03_aymeric


In [3]:
best_model_path = bdd.iloc[0,:].model_path
print(best_model_path)

best_model_class = pd.read_csv(best_model_path+"/cingulate_ACCpatterns_embeddings/cross_val_predicted_labels.csv", index_col=0)
best_model_class

/neurospin/dico/agaudin/Runs/04_pointnet/Output/pointnet/fold_30_T=0.5


,label,predicted_0,predicted_1,predicted_2,predicted_3,predicted_4,predicted_5,predicted_6,predicted_7,predicted_8,predicted_9,predicted_10,predicted_11,predicted_12,predicted_13,predicted_14,predicted_15,predicted_16,predicted_17,predicted_18,predicted_19,predicted_20,predicted_21,predicted_22,predicted_23,predicted_24,predicted_25,predicted_26,predicted_27,predicted_28,predicted_29,predicted_30,predicted_31,predicted_32,predicted_33,predicted_34,predicted_35,predicted_36,predicted_37,predicted_38,...,predicted_212,predicted_213,predicted_214,predicted_215,predicted_216,predicted_217,predicted_218,predicted_219,predicted_220,predicted_221,predicted_222,predicted_223,predicted_224,predicted_225,predicted_226,predicted_227,predicted_228,predicted_229,predicted_230,predicted_231,predicted_232,predicted_233,predicted_234,predicted_235,predicted_236,predicted_237,predicted_238,predicted_239,predicted_240,predicted_241,predicted_242,predicted_243,predicted_244,predicted_245,predicted_246,predicted_247,predicted_248,predicted_249,median_pred,mean_pred
Subject,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1-03HUBJO_t0,1,1.085944,1.051383,1.077067,1.124209,1.035375,1.064797,1.131702,1.066546,1.158811,1.143650,0.939495,1.229587,0.967766,1.029541,1.048254,0.997106,1.137583,0.946455,1.108872,1.056414,1.088654,1.247905,0.978834,0.801957,1.137886,1.041643,1.178530,0.908023,1.145561,1.312186,1.030315,1.027709,1.068064,1.025060,1.104587,1.135364,0.968597,1.077847,1.178485,...,1.187734,1.067969,1.176935,1.082439,0.958146,0.916462,1.055569,1.060398,1.074254,1.149017,1.086975,0.931681,1.090945,1.058901,1.262633,1.063964,1.044004,1.074569,1.218147,1.056606,1.087445,1.308536,1.161819,1.187327,1.146723,1.131997,0.944030,0.879783,0.780474,1.143416,0.923405,1.115958,1.343148,1.103910,1.046159,1.142645,1.050461,0.976973,1.074412,1.079220
1-08ANDTI_t0,1,0.370679,0.834513,0.581987,0.378756,0.583937,0.522597,0.609104,0.690729,0.495865,0.634191,0.405676,0.564209,0.540698,0.246121,0.497119,0.650953,0.423078,0.427186,0.505291,0.399299,0.650202,0.591936,0.509628,0.388335,0.511616,0.606911,0.602368,0.578545,0.640482,0.798435,0.610099,0.436703,0.538814,0.260495,0.619187,0.499188,0.600363,0.557233,0.762378,...,0.837629,0.540209,0.755698,0.542654,0.530261,0.293028,0.485079,0.625285,0.791106,0.603021,0.581565,0.490532,0.838052,0.552748,0.673264,0.436627,0.793870,0.542150,0.720006,0.700397,0.584582,0.647555,0.607201,0.551537,0.623616,0.482203,0.510876,0.503548,0.408139,0.457622,0.392902,0.650919,0.756105,0.669078,0.485912,0.697749,0.425793,0.267314,0.566727,0.569732
1-11LEBJO_t0,1,0.379897,0.850434,0.591653,0.379307,0.589350,0.522810,0.617991,0.703430,0.493283,0.634008,0.413860,0.565250,0.550928,0.250164,0.506170,0.664095,0.426897,0.444318,0.509626,0.403656,0.661920,0.592983,0.530085,0.403119,0.512967,0.616393,0.609285,0.594167,0.647467,0.796401,0.620915,0.448625,0.544788,0.262103,0.629493,0.503615,0.614171,0.569878,0.765343,...,0.844946,0.547147,0.760870,0.553052,0.546463,0.301296,0.487138,0.629777,0.801555,0.609932,0.588120,0.500774,0.847454,0.564022,0.675403,0.440878,0.804632,0.549949,0.722936,0.711388,0.585078,0.639369,0.611450,0.562362,0.629158,0.485996,0.524212,0.522457,0.423470,0.459682,0.407656,0.658867,0.755291,0.678911,0.493181,0.706374,0.436636,0.272810,0.572766,0.577394
1-15LEHMI_t0,0,0.838717,1.045822,0.873905,0.723710,0.779527,0.691132,0.870201,0.898502,0.665945,0.694860,0.774142,0.802358,0.833274,0.701885,0.843790,0.922554,0.765271,0.921073,0.787055,0.764674,0.910224,0.752177,0.978544,0.829924,0.762062,0.847359,0.853625,0.912063,0.865953,0.815801,0.861423,0.844964,0.824325,0.679569,0.894092,0.812528,0.925632,0.916581,0.871274,...,0.910346,0.855308,0.867298,0.849715,0.933427,0.742171,0.711617,0.764123,0.879687,0.869390,0.839057,0.833937,0.959402,0.894434,0.835123,0.770395,0.913839,0.819185,0.840204,0.922394,0.755094,0.716573,0.807133,0.915344,0.800323,0.803878,0.875491,0.930140,0.809801,0.740534,0.878937,0.885167,0

In [4]:
densenet_bdd = bdd[bdd.backbone_name == 'densenet']

best_densenet_path = densenet_bdd.iloc[0,:].model_path
print(best_densenet_path)

best_densenet_class = pd.read_csv(best_densenet_path+"/cingulate_ACCpatterns_embeddings/cross_val_predicted_labels.csv", index_col=0)
best_densenet_class

/neurospin/dico/agaudin/Runs/04_pointnet/Output/2022-09-02/09-28-35


,label,predicted_0,predicted_1,predicted_2,predicted_3,predicted_4,predicted_5,predicted_6,predicted_7,predicted_8,predicted_9,predicted_10,predicted_11,predicted_12,predicted_13,predicted_14,predicted_15,predicted_16,predicted_17,predicted_18,predicted_19,predicted_20,predicted_21,predicted_22,predicted_23,predicted_24,predicted_25,predicted_26,predicted_27,predicted_28,predicted_29,predicted_30,predicted_31,predicted_32,predicted_33,predicted_34,predicted_35,predicted_36,predicted_37,predicted_38,...,predicted_212,predicted_213,predicted_214,predicted_215,predicted_216,predicted_217,predicted_218,predicted_219,predicted_220,predicted_221,predicted_222,predicted_223,predicted_224,predicted_225,predicted_226,predicted_227,predicted_228,predicted_229,predicted_230,predicted_231,predicted_232,predicted_233,predicted_234,predicted_235,predicted_236,predicted_237,predicted_238,predicted_239,predicted_240,predicted_241,predicted_242,predicted_243,predicted_244,predicted_245,predicted_246,predicted_247,predicted_248,predicted_249,median_pred,mean_pred
Subject,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1-03HUBJO_t0,1,1.007062,0.653706,1.809915,1.799068,0.859552,1.412365,0.915745,1.419833,0.783324,0.846208,0.645754,0.983092,1.668858,1.688191,0.827004,1.573648,1.650643,1.711957,1.223996,0.254083,0.756636,1.646342,1.522017,1.549304,0.888340,1.744299,0.520394,1.695789,1.751260,1.643586,1.252744,2.063559,1.002434,1.504885,0.729904,1.619056,1.491390,0.740622,2.215121,...,1.851784,1.903306,0.603463,1.317793,1.547781,0.842388,1.444718,0.806529,1.716713,1.466112,0.671280,0.887966,1.396197,1.185032,1.834398,0.902342,0.893137,0.380356,1.586407,1.150849,1.334586,1.076999,1.209491,0.982987,1.794889,1.324006,1.325089,0.945519,0.716732,0.995926,0.876850,1.255173,1.481667,1.602663,1.479653,1.597264,1.260885,0.847639,1.189575,1.193186
1-08ANDTI_t0,1,0.452632,-0.063741,1.309122,1.132577,0.398506,0.851268,0.217811,0.652256,-0.017028,0.045408,-0.304269,0.335750,0.807476,1.154228,0.054814,1.047445,1.051519,1.298275,0.675359,-0.290942,0.004812,0.529540,1.256134,1.016232,0.257857,0.971476,-0.021746,1.194739,0.960567,0.908536,0.624189,1.169692,0.275013,0.638367,0.187772,0.833255,0.885820,0.051843,1.495946,...,1.180895,1.025375,-0.161066,0.618451,0.721884,0.275385,0.778561,0.251231,1.019783,0.842863,-0.150585,0.192447,1.101013,0.388985,1.058503,0.255981,-0.035952,-0.326101,0.929093,0.477855,0.475440,0.315918,0.503417,0.312580,1.161673,0.783396,0.483991,0.366437,0.279207,0.109975,0.143547,0.793274,1.052016,1.154874,1.048417,0.913148,0.410028,0.313450,0.483814,0.515278
1-11LEBJO_t0,1,0.795762,0.290454,1.612666,1.499291,0.684621,1.179280,0.582689,1.043915,0.399384,0.451023,0.153340,0.715228,1.249647,1.470815,0.443008,1.358478,1.376911,1.575323,1.005537,0.046418,0.416059,1.028716,1.475601,1.347981,0.610221,1.379971,0.295593,1.508456,1.398027,1.321134,0.967584,1.615489,0.670740,1.085355,0.511829,1.253328,1.209071,0.396701,1.895450,...,1.578261,1.459074,0.233175,1.021953,1.130886,0.596111,1.127633,0.563560,1.380131,1.208787,0.250620,0.559807,1.344094,0.796021,1.474416,0.637749,0.416605,0.012126,1.306684,0.836724,0.931841,0.726643,0.865593,0.671454,1.521388,1.122955,0.916858,0.700847,0.542147,0.553928,0.519386,1.108074,1.302403,1.469597,1.322647,1.304520,0.858288,0.626007,0.857313,0.884950
1-15LEHMI_t0,0,0.543701,0.197425,1.419454,1.302202,0.463933,0.957562,0.425637,0.878724,0.236308,0.243375,-0.002165,0.470614,1.219356,1.255923,0.290908,1.156977,1.208339,1.392689,0.791635,-0.233873,0.229770,1.026244,1.259318,1.059976,0.346688,1.210809,0.063355,1.309256,1.213795,1.157983,0.796247,1.458090,0.436373,0.966966,0.247103,1.041749,1.095231,0.256105,1.689244,...,1.305349,1.368598,0.032017,0.731464,1.007279,0.364231,1.035146,0.347745,1.293148,0.947197,0.139192,0.350613,1.063977,0.587127,1.292477,0.316170,0.282749,-0.097526,1.117533,0.691108,0.721269,0.494571,0.725495,0.519297,1.304248,0.878850,0.739205,0.540067,0.335638,0.331647,0.32849

In [10]:
def get_classifier_by_quality(labels_df, q=0.5):
    aucs = labels_df.apply(compute_auc, args=[labels_df.label])
    aucs = aucs[~aucs.index.isin(['label', 'median_pred', 'mean_pred'])]
    model_auc = aucs.quantile(q=q, interpolation='nearest')
    aucs = aucs[aucs == model_auc]
    return(aucs.index[0], model_auc)

In [17]:
def get_subjects_to_visualize(model_path, q=1):
    model_classifiers = pd.read_csv(model_path+"/cingulate_ACCpatterns_embeddings/cross_val_predicted_labels.csv", index_col=0)

    classifier_name, _ = get_classifier_by_quality(model_classifiers, q=q)

    preds = model_classifiers[['label', classifier_name]]
    preds.sort_values(by=classifier_name, ascending=False, inplace=True)

    subjects_tag = ["worst 1 pred", "best 0 pred", "limit 0 pred",
                    "best 1 pred", "worst 0 pred", "limit 1 pred"]
    subjects_to_visualize = []
    predictions = []

    for i in range(2):
        half_preds = preds[preds.label == i]

        subjects_to_visualize.append(half_preds.index[0])
        subjects_to_visualize.append(half_preds.index[-1])

        predictions.append(half_preds[classifier_name][0])
        predictions.append(half_preds[classifier_name][-1])

        if i == 0:
            half_preds = half_preds[half_preds[classifier_name] <= 0.5]
            if half_preds.values != []:
                subjects_to_visualize.append(half_preds.index[0])
                predictions.append(half_preds[classifier_name][0])

        if i == 1:
            half_preds = half_preds[half_preds[classifier_name] >= 0.5]
            if half_preds.values != []:
                subjects_to_visualize.append(half_preds.index[-1])
                predictions.append(half_preds[classifier_name][-1])

    return pd.DataFrame({'tag': subjects_tag, 'subject':subjects_to_visualize, 'preds': predictions})

In [18]:
subjects_to_visualize = get_subjects_to_visualize("/neurospin/dico/agaudin/Runs/04_pointnet/Output/2022-09-30/17-27-31")
subjects_to_visualize

/neurospin/dico/agaudin/Runs/04_pointnet/2022_jchavas_cingulate_inhibitory_control/venv_local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/neurospin/dico/agaudin/Runs/04_pointnet/2022_jchavas_cingulate_inhibitory_control/venv_local/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/neurospin/dico/agaudin/Runs/04_pointnet/2022_jchavas_cingulate_inhibitory_control/venv_local/lib/python3.6/site-packages/ipykernel_launcher.py:31: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


,tag,subject,preds
0,worst 1 pred,sub-inhibition03_ses-pretraining_T1w,1.228063
1,best 0 pred,sub-active12_ses-pretraining_T1w,-0.462505
2,limit 0 pred,nih_chp_01503_t1,0.495737
3,best 1 pred,nih_chp_04815_t1,1.216350
4,worst 0 pred,sub-active06_ses-pretraining_T1w,-0.335271
5,limit 1 pred,nih_chp_03896_t1,0.513795
